In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

In [9]:
class EmojiTextClassifier:
    def __init__(self,vector_shape):
        self.vector_shape = vector_shape

    def load_dataset(self,dataset_path):
        self.df = pd.read_csv(dataset_path)
        self.X = np.array(self.df['sentence'],dtype=object)
        self.Y = np.array(self.df['label'],dtype=int)
        return self.X,self.Y

    def load_features_vector(self,file_txt_path):
        self.f = open(file_txt_path,encoding='utf-8')
        self.word_vectors = {}
        for line in self.f:
            line = line.strip().split()
            word = line[0]
            vector = np.array(line[1:],dtype=np.float64)
            self.word_vectors[word] = vector

        return self.word_vectors


    def sentence_to_feature_vectors_avg(self,sentence):
          self.sentence = sentence.lower()
          words = self.sentence.strip().split(' ')
          sum_vectors = np.zeros((self.vector_shape,))
          for word in words:
              sum_vectors += self.word_vectors[word]

          avg_words = sum_vectors / len(words)

          return avg_words


    def preprocess(self,X,Y):
        self.X_avg = []
        self.X = X
        self.Y = Y
        for x in X:
            self.X_avg.append(self.sentence_to_feature_vectors_avg(x))


        self.X_avg = np.array(self.X_avg).astype('float32')
        self.Y_one_hot = tf.keras.utils.to_categorical(self.Y,num_classes=5)

        return self.X_avg,self.Y_one_hot

    def load_model(self):

        model = tf.keras.models.Sequential([
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Dense(5,input_shape=(self.vector_shape,),activation='softmax')
        ])
        return model

    def train(self,X_train,Y_train,epochs):
        #self.input_shape =input_shape
        self.X_train_avg,self.Y_train_one_hot = self.preprocess(X_train,Y_train)
        print(self.X_train_avg.shape,self.Y_train_one_hot.shape)
        self.model = self.load_model()
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        #self.model.fit(self.X_train_avg,self.Y_train_one_hot,epochs)
        return self.model

    def test(self,model,X_test,Y_test):
        self.X_test = X_test
        self.Y_test = Y_test
        #self.X_test_avg,self.Y_test_one_hot = self.preprocess(self.X_test,self.Y_test)
        self.X_avg = []
        self.new_Y_test = []
        i = 0

        for index,x in enumerate(self.X_test):
            if x.endswith('\t') == True:
              self.X_avg.append(self.sentence_to_feature_vectors_avg(x))
              self.new_Y_test.append(self.Y_test[index])



        self.X_avg = np.array(self.X_avg)
        self.new_Y_test = np.array(self.new_Y_test)
        self.Y_one_hot = tf.keras.utils.to_categorical(self.new_Y_test,num_classes=5)
        print(np.shape(self.X_avg),np.shape(self.Y_one_hot))

        accuracy,loss = model.evaluate(self.X_avg,self.Y_one_hot)

        return accuracy,loss

    def label_to_emoji(self,label):
        self.label = label
        emojies=['❤️','⚾','😊','😞','🍴']

        return emojies[self.label]

    def predict(self,sentece_test):
        start_time = time.time()
        self.sentence_test = sentece_test
        self.my_test_avg = self.sentence_to_feature_vectors_avg(self.sentence_test)
        self.my_test_avg = np.array([self.my_test_avg])
        self.result = self.model.predict(self.my_test_avg)
        y_pred = np.argmax(self.result)

        return self.label_to_emoji(y_pred),time.time() - start_time

## 50 Dimention

In [10]:
emoji_text_50d = EmojiTextClassifier(50)

In [11]:
X_train,Y_train = emoji_text_50d.load_dataset('/content/drive/MyDrive/Emoji_Text_Classification/train.csv')
X_test,Y_test= emoji_text_50d.load_dataset('/content/drive/MyDrive/Emoji_Text_Classification/test.csv')

emoji_text_50d.load_features_vector('/content/drive/MyDrive/Emoji_Text_Classification/glove-6B/glove.6B.50d.txt')
X_train_avg,Y_train_one_hot = emoji_text_50d.preprocess(X_train,Y_train)
model=emoji_text_50d.train(X_train,Y_train,epochs=300)
model.fit(X_train_avg,Y_train_one_hot,epochs=300)

(132, 50) (132, 5)
Epoch 1/300
5/5 [==============================] - 3s 13ms/step - loss: 1.8070 - accuracy: 0.1742
Epoch 2/300
5/5 [==============================] - 0s 4ms/step - loss: 1.7692 - accuracy: 0.1818
Epoch 3/300
5/5 [==============================] - 0s 4ms/step - loss: 1.7442 - accuracy: 0.1818
Epoch 4/300
5/5 [==============================] - 0s 4ms/step - loss: 1.7418 - accuracy: 0.2273
Epoch 5/300
5/5 [==============================] - 0s 4ms/step - loss: 1.7587 - accuracy: 0.1970
Epoch 6/300
5/5 [==============================] - 0s 3ms/step - loss: 1.7360 - accuracy: 0.2500
Epoch 7/300
5/5 [==============================] - 0s 3ms/step - loss: 1.7081 - accuracy: 0.2273
Epoch 8/300
5/5 [==============================] - 0s 3ms/step - loss: 1.6828 - accuracy: 0.2273
Epoch 9/300
5/5 [==============================] - 0s 3ms/step - loss: 1.6647 - accuracy: 0.2803
Epoch 10/300
5/5 [==============================] - 0s 3ms/step - loss: 1.6496 - accuracy: 0.2652
Epoch 11/

In [19]:
accuracy,loss = emoji_text_50d.test(model,X_test,Y_test)


(49, 50) (49, 5)
2/2 [==============================] - 0s 6ms/step - loss: 0.7332 - accuracy: 0.7551


In [20]:
infrence = 'the Weather is Sunny'
emoji,timer = emoji_text_50d.predict(infrence)
emoji,timer

1/1 [==============================] - 0s 17ms/step


('😊', 0.06578493118286133)

## 100 Dimention

In [14]:
emoji_text_100d = EmojiTextClassifier(100)

In [ ]:
X_train,Y_train = emoji_text_100d.load_dataset('/content/drive/MyDrive/Emoji_Text_Classification/train.csv')
X_test,Y_test = emoji_text_100d.load_dataset('/content/drive/MyDrive/Emoji_Text_Classification/test.csv')

emoji_text_100d.load_features_vector('/content/drive/MyDrive/Emoji_Text_Classification/glove-6B/glove.6B.100d.txt')
X_train_avg,Y_train_one_hot = emoji_text_100d.preprocess(X_train,Y_train)
model_100 = emoji_text_100d.train(X_train,Y_train,epochs=300)
model_100.fit(X_train_avg,Y_train_one_hot,epochs=300)

In [31]:
eval = emoji_text_100d.test(model_100,X_test,Y_test)
eval

(49, 100) (49, 5)
2/2 [==============================] - 0s 9ms/step - loss: 0.6593 - accuracy: 0.7959


(0.6593421697616577, 0.795918345451355)

In [18]:
infrence = 'the Weather is Sunny'
emoji,timer = emoji_text_100d.predict(infrence)
emoji,timer

1/1 [==============================] - 0s 46ms/step


('😊', 0.09525656700134277)

## 200 Dimention

In [21]:
emoji_text_200d = EmojiTextClassifier(200)

In [22]:
X_train,Y_train = emoji_text_200d.load_dataset('/content/drive/MyDrive/Emoji_Text_Classification/train.csv')
X_test,Y_test = emoji_text_200d.load_dataset('/content/drive/MyDrive/Emoji_Text_Classification/test.csv')

In [ ]:
emoji_text_200d.load_features_vector('/content/drive/MyDrive/Emoji_Text_Classification/glove-6B/glove.6B.200d.txt')
X_train_avg,Y_train_one_hot = emoji_text_200d.preprocess(X_train,Y_train)
model_200 = emoji_text_200d.train(X_train,Y_train,epochs=300)
model_200.fit(X_train_avg,Y_train_one_hot,epochs=300)

In [24]:
loss,accuracy= emoji_text_200d.test(model_200,X_test,Y_test)
loss,accuracy

(49, 200) (49, 5)
2/2 [==============================] - 0s 8ms/step - loss: 0.5209 - accuracy: 0.8367


(0.5209062695503235, 0.8367347121238708)

In [25]:
infrence = 'the Weather is Sunny'
emoji,timer = emoji_text_200d.predict(infrence)
emoji,timer

1/1 [==============================] - 0s 36ms/step


('😊', 0.08133244514465332)

## 300 Dimention

In [26]:
emoji_text_300d = EmojiTextClassifier(300)

In [27]:
X_train,Y_train = emoji_text_300d.load_dataset('/content/drive/MyDrive/Emoji_Text_Classification/train.csv')
X_test,Y_test = emoji_text_300d.load_dataset('/content/drive/MyDrive/Emoji_Text_Classification/test.csv')

In [ ]:
emoji_text_300d.load_features_vector('/content/drive/MyDrive/Emoji_Text_Classification/glove-6B/glove.6B.300d.txt')
X_train_avg,Y_train_one_hot = emoji_text_300d.preprocess(X_train,Y_train)
model_300 = emoji_text_300d.train(X_train,Y_train,epochs=300)
model_300.fit(X_train_avg,Y_train_one_hot,epochs=300)

In [29]:
loss,accuracy= emoji_text_300d.test(model_300,X_test,Y_test)
loss,accuracy

(49, 300) (49, 5)
2/2 [==============================] - 0s 8ms/step - loss: 0.4601 - accuracy: 0.8980


(0.46014684438705444, 0.8979591727256775)

In [30]:
infrence = 'I workout in gym'
emoji,timer = emoji_text_300d.predict(infrence)
emoji,timer

1/1 [==============================] - 0s 35ms/step


('⚾', 0.07652163505554199)